# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686572


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:07<01:55,  3.99s/it]

Rendering models:  10%|▉         | 3/31 [00:09<01:29,  3.18s/it]

Rendering models:  13%|█▎        | 4/31 [00:10<01:11,  2.65s/it]

Rendering models:  16%|█▌        | 5/31 [00:10<00:50,  1.94s/it]

...

Rendering models:  90%|█████████ | 28/31 [00:29<00:01,  1.51it/s]

Rendering models:  94%|█████████▎| 29/31 [00:29<00:01,  1.50it/s]

Rendering models:  97%|█████████▋| 30/31 [00:30<00:00,  1.28it/s]

Rendering models: 100%|██████████| 31/31 [00:31<00:00,  1.20it/s]

BridieK                               0.000453
QuiteAwful                            0.001350
tjlovesfunhotmail.com                 0.009057
not-logged-in-f7e2260ed4a9517a3e4a    0.000514
Staceyann116                          0.004988
asucich                               0.000534
not-logged-in-f7e2260ed4a9517a3e4a    0.077700
thedarktree                           0.000405
not-logged-in-c3e2bb63f9ec3bdeea91    0.116131
alondra_                              0.012101
not-logged-in-c08c0c43bd0e8eef780a    0.002337
not-logged-in-243ecd760a5ad294f57a    0.014015
not-logged-in-1df2dc33157510bbec3e    0.000647
not-logged-in-5f30b2bb9b7f9d49da41    0.003997
BirdLeaf                              0.001825
pangeli5                              0.000490
Lavadude                              0.006317
ShirleyL                              0.007865
ShirleyL                              0.000573
AspenTreez                            0.001333
jiipee                                0.000435
nicholas.    

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())